In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [2]:
data = data.drop(columns = ["ID","ZIP Code"])

In [3]:
X = data.drop("Personal Loan",axis=1)
y = data["Personal Loan"]

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [4]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.optimizers import Adam

optimizer = Adam(lr=0.01)
loss = 'binary_crossentropy'

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))


C:\Users\Srivatsan\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [6]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(11, 6) dtype=float32, numpy=
 array([[ 0.0146755 , -0.19158009, -0.5307023 ,  0.55748904, -0.01115543,
         -0.34351382],
        [ 0.3726123 ,  0.54244256, -0.52558905, -0.31096554,  0.39799362,
          0.16105402],
        [ 0.00529855, -0.46202874,  0.34736222, -0.1484026 , -0.05151963,
          0.02318054],
        [ 0.08752823, -0.2783596 , -0.31275603, -0.52396244,  0.4094404 ,
          0.41764987],
        [-0.5848882 , -0.13736147, -0.40761465, -0.5537429 , -0.5271365 ,
         -0.5910771 ],
        [-0.3714926 ,  0.11522615,  0.33759505, -0.0879792 ,  0.22152317,
         -0.40541238],
        [ 0.33830154,  0.28104752,  0.0686416 , -0.02141482, -0.43631595,
          0.532251  ],
        [ 0.4426899 ,  0.38657957, -0.48502088,  0.13759392, -0.09299845,
         -0.33203545],
        [-0.04727393, -0.41171587,  0.06225497, -0.5814186 , -0.38542065,
         -0.00240737],
        [ 0.16530329,  0.05423975, -0.14267603, -0.31271765,

In [7]:
def fitness(weights):
    w0 = weights[:66].reshape((11, 6))
    w1 = weights[66:72].reshape((6,1))
    b0= np.array([0.,0.,0.,0.,0.,0.])
    b1 = np.array([0.])


    model.layers[0].set_weights([w0, b0])
    model.layers[1].set_weights([w1, b1])
    
    
    model.compile(optimizer=optimizer, loss=loss)
    
    Loss = model.evaluate(X_train, y_train, verbose=0)
    return -Loss

In [3]:
def pso(fitness, num_weights, max_iter, pop_size, w, c1, c2):
    
    swarm_pos = np.random.uniform(low=-1, high=1, size=(pop_size, num_weights))
    swarm_vel = np.zeros((pop_size, num_weights))
    swarm_best_pos = swarm_pos.copy()
    swarm_best_fitness = np.zeros(pop_size)
    global_best_pos = swarm_pos[0].copy()
    global_best_fitness = float('inf')
    prev_gbf = 0
    
    
    for i in range(max_iter):
        for j in range(pop_size):
            # Update velocity
            swarm_vel[j] = (w * swarm_vel[j] 
                            + c1 * np.random.uniform(0, 1, num_weights) * (swarm_best_pos[j] - swarm_pos[j])
                            + c2 * np.random.uniform(0, 1, num_weights) * (global_best_pos - swarm_pos[j]))
            
            swarm_pos[j] += swarm_vel[j]
            
            swarm_pos[j] = np.clip(swarm_pos[j], -1, 1)
            
            swarm_fitness = fitness(swarm_pos[j])
            
            # Update personal best
            if swarm_fitness < swarm_best_fitness[j]:
                swarm_best_fitness[j] = swarm_fitness
                swarm_best_pos[j] = swarm_pos[j].copy()
                
                # Update global best
                if swarm_fitness < global_best_fitness:
                    global_best_fitness = swarm_fitness
                    global_best_pos = swarm_pos[j].copy()
        if (i%10==0):
            if (prev_gbf == global_best_fitness):
                print(f"Iteration {i+1}: Best Fitness = {-global_best_fitness:.4f} CONVERGED BREAKING")
            else:
                print(f"Iteration {i+1}: Best Fitness = {-global_best_fitness:.4f}")
            prev_gbf = global_best_fitness
            
    
    return global_best_pos

In [42]:
num_weights = 72
max_iter = 50
pop_size = 10
w = 0.5
c1 = 2
c2 = 2
best_weights = pso(fitness, num_weights, max_iter, pop_size, w, c1, c2)

Iteration 1: Best Fitness = 1.2536
Iteration 11: Best Fitness = 3.3867
Iteration 21: Best Fitness = 3.3867
Iteration 31: Best Fitness = 3.3867
Iteration 41: Best Fitness = 3.3867


In [47]:
w0 = best_weights[:66].reshape((11, 6))
w1 = best_weights[66:72].reshape((6,1))
b0= np.array([0.,0.,0.,0.,0.,0.])
b1 = np.array([0.])


model.layers[0].set_weights([w0, b0])
model.layers[1].set_weights([w1, b1])
    
    
model.compile(optimizer=optimizer, loss=loss)

In [59]:
y_pred = model.predict(X_test)
y_pred

47/47 [==============================] - 0s 716us/step


array([[0.96012574],
       [0.997856  ],
       [0.95901936],
       ...,
       [0.9999977 ],
       [0.70206517],
       [0.99823475]], dtype=float32)

In [62]:
y_pred = (y_pred < 0.05)


In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7706666666666667